In [ ]:
# Standard library imports
import time
from datetime import datetime



# Third party imports
import streamlit as st
import altair as alt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.simplefilter('ignore') # We sometimes get warnings in cell outputs, stop these from being displayed.

from snowflake.snowpark import Session, context, functions
from snowflake.snowpark.types import DateType, FloatType, IntegerType, StringType, StructField, StructType, Variant
from snowflake.ml import dataset, registry
from snowflake.ml.modeling import pipeline, preprocessing, metrics, model_selection, xgboost
from snowflake.core import Root, CreateMode, task



# Local application imports
# None



# Get a Snowflake session, this is where you would normally login, instead just use get_active_session!
sf_session = get_active_session()



# Specify which DB, Schema and Tables we'll be using.
db_name = 'INSURANCE_2'
schema_name = 'PUBLIC'
client_raw_data_table_name = db_name + '.' + schema_name + '.' + 'CLIENTS'
client_features_table_name = db_name + '.' + schema_name + '.' + 'CLIENT_FEATURES'


In [ ]:
# Features shown here:
# 📘 Snowflake Notebooks is a development interface offering a cell-based programming environment.
#    https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks

# 📘 Snowpark API provides an intuitive library for querying and processing data at scale in Snowflake.
#    https://docs.snowflake.com/en/developer-guide/snowpark/index

# 📘 Streamlit is a Python library that makes it easy to create custom web apps for machine learning and data science.
#    https://docs.snowflake.com/en/developer-guide/streamlit/about-streamlit



# 🦜 What does the data we'll be working with look like? Let's use the Snowpark API to see.



# To create a DataFrame stored in Snowflake we can simply call the table method.
clients_df = sf_session.table( client_raw_data_table_name )



# You can use Streamlit components in Notebooks since they all run on Snowflake.
st.dataframe( clients_df.first(10) )



# 🍾 Notebooks don't display tables of data nicely, meaning you often end up importing
#    other libraries. With Notebooks & Streamlit you no longer need to do that, reducing dependencies, 
#    complexity and ultimately saving you development time.

In [ ]:
# Features shown here:
# 📘 Snowpark ML Modeling API for feature engineering and preprocessing
#    https://docs.snowflake.com/en/developer-guide/snowflake-ml/modeling
#    https://docs.snowflake.com/en/developer-guide/snowpark-ml/reference/latest/modeling


# 🦜 The Snowpark ML Modeling API uses familiar Python frameworks such as scikit-learn and XGBoost for 
#    preprocessing data, feature engineering, and training models inside Snowflake. We need to encode 
#    some of the values in our dataset so they can be used by our model later. To do this we'll use 
#    the snowflake.ml.modeling.preprocessing.OrdinalEncoder, which is based on scikit's  encoder 
#    sklearn.preprocessing.OrdinalEncoder, so it should be what you know and love!



# Let's turn the gender and smoker fields into 1's and 0's by creating an encoder.
ordinal_encoder = preprocessing.OrdinalEncoder(
    input_cols = ['GENDER', 'SMOKER', 'COUNTY'],
    output_cols = ['GENDER_ENCODED', 'SMOKER_ENCODED', 'COUNTY_ENCODED'],
    encoded_missing_value = '999',
    drop_input_cols = True
)



# Now we can apply it to the dataframe which holds our raw client data, and create a new dataframe
# which has the original columns, which may hold PII data, replaced with encoded values.
client_features_df = ordinal_encoder.fit( clients_df ).transform( clients_df )



# The Gender and Smoker fields are now 0 or 1 instead of text.
st.dataframe( client_features_df.first(10) )



# 🍾 These ready-to-use ML Functions, based on common libraries such as sklearn, can help shorten 
#    development time and democratize ML across your organization.

In [ ]:
# Features shown here:
# 📘 Snowpark ML Modeling API for feature engineering and preprocessing
#    https://docs.snowflake.com/en/developer-guide/snowflake-ml/modeling
#    https://docs.snowflake.com/en/developer-guide/snowpark-ml/reference/latest/modeling

# 📘 Snowpark API provides an intuitive library for querying and processing data at scale in Snowflake.
#    https://docs.snowflake.com/en/developer-guide/snowpark/index



# 🦜 We want to regularly update the data our models use for training and fitting, and so we'll
#    automate this by creating a Python function locally which is then saved as a Stored Procedure
#    in Snowflake so it can be called whenever we, or anyone else, needs it.



# Let's define the Python function locally first.
def encodeColumns( 
    session: Session, 
    raw_table: str, 
    processed_table: str, 
    input_cols: list, 
    output_cols: list, 
    encoded_missing_value: int 
) -> str:

    # Third party imports
    from snowflake.ml.modeling import preprocessing, pipeline
    
    # With scikit-learn, it is common to run a series of transformations using a "pipeline". 
    # scikit-learn pipelines do not work with Snowpark ML classes, so Snowpark ML provides a 
    # Snowpark Python version of sklearn.pipeline.Pipeline for running a series of transformations. 
    # This class is in the snowflake.ml.modeling.pipeline package, and it works the same as 
    # the scikit-learn version.
    preprocessing_pipeline = pipeline.Pipeline(
        steps=[
            (
                'OrdinalEncoding',
                preprocessing.OrdinalEncoder( 
                    input_cols = input_cols,
                    output_cols = output_cols,
                    encoded_missing_value = encoded_missing_value,
                    drop_input_cols = True
                )
            )
        ]
    )

    # Where are we getting the data from?
    df = session.table( raw_table )

    # Apply the pipeline.
    features_df = preprocessing_pipeline.fit( df ).transform( df )

    # Write the table to Snowflake.
    features_df.write.mode( 'overwrite' ).save_as_table( processed_table )

    # Provide a message back.
    print( f'Feature Engineering done, table created: {processed_table}' )
    
    return processed_table



# Create a Stored Procedure which uses the Python function we created above.
stored_procedure_object = functions.sproc(
    name = 'ENCODE_COLUMNS',
    func = encodeColumns,
    is_permanent = True,
    stage_location = '@ML_MODELS',
    replace = True,
    packages = ['snowflake-snowpark-python', 'snowflake-ml-python'],
)



# 🍾 The Snowpark ML Modeling API allows you to build the ML pipelines using the libraries 
#    you know and love without needing to learn anything new.

# 🍾 Python developers can build locally then push their work into Snowflake for others
#    to use who may be more familiar using SQL. No need to re-write Python into SQL and
#    vice versa, saving you time, effort and development cost.

In [ ]:
# Features shown here:
# 📘 Snowflake Python API provides comprehensive APIs for interacting with Snowflake resources across 
#    data engineering, Snowpark, Snowpark ML, and application workloads using a first-class Python API.
#    https://docs.snowflake.com/en/developer-guide/snowflake-python-api/reference/latest/index



# 🦜 We're going to use a Task to schedule the Stored Procedure to run every day so we have up-to-date
#    data to use in our models. We'll do this by creating the Task through the Python API.



# This is the entry point of the Snowflake Core Python APIs that manage the Snowflake objects.
root = Root( sf_session )



# This is where we'll store the Task, its like setting the context in Worksheets.
tasks_context = root.databases[db_name].schemas[schema_name].tasks



# Create a Task in Snowflake, scheduled to run at 10am every day.
tasks_context.create( 
    task.Task(
        'CALL_ENCODE_COLUMNS',
        task.StoredProcedureCall( stored_procedure_object ),
        warehouse = 'COMPUTE_WH',
        schedule = task.Cron( '10 * * * *', 'Europe/London' ),
    ),
    mode = CreateMode.or_replace,
)



# What did we create?
st.dataframe( sf_session.sql( 'show tasks' ) )



# 🦜 As we've seen, Tasks can be defined to automate data engineering workflows, specifically 
#    recurring tasks, natively in Snowflake. However, writing a full-scale pipeline with many 
#    tasks has been hard. Often, customers have built their own framework on top of Snowflake 
#    to solve a number of programmability issues such as constructing logical groups of tasks, 
#    retries, dynamic behavior, configurability, logging, etc.

#    To start addressing these needs, Snowflake introduced the following features:
#    + Richer Task Graph Features: Finalizer Tasks, Runtime Reflection Variables, Task 
#      Predecessor Return Values, etc.
#    + SNOWFLAKE PYTHON API: Provides Python APIs for Snowflake resources.
#    + Enhanced Task UI in Snowsight: Enables users to easily manage and monitor all tasks 
#      from a scheduled graph in one place.
#    + Snowpark ML API: Python API for preprocessing, feature engineering, and training models
#      following familiar Python frameworks such as scikit-learn and XGBoost.
#    + Snowpark Model Registry: Snowpark Model Registry allows customers to securely manage 
#      models and their metadata in Snowflake, regardless of origin.

# 🔗 https://medium.com/snowflake/productionize-your-ml-workflow-using-snowflake-task-dag-apis-8470aa33172c

In [ ]:
# 🦜 Rather than waiting for a day, let's call the sproc directly to engineer the features now.



# Use Snowpark to call the Stored Procedure.
sf_session.call( 
    'ENCODE_COLUMNS', # The sproc name we want to call.
    client_raw_data_table_name, # The table holding our raw data.
    client_features_table_name, # The table we'll create which will have our engineered data.
    ['GENDER', 'SMOKER', 'COUNTY'], # Columns to encode.
    ['GENDER_ENCODED', 'SMOKER_ENCODED', 'COUNTY_ENCODED'], # Column names added to the raw data with encoded data.
    999
)



# What have we created?
client_features_df = sf_session.table( client_features_table_name )



# You can use Streamlit components in Notebooks since they all run on Snowflake.
st.dataframe( client_features_df.first(10) )

In [ ]:
# Features shown here:
# 📘 Datasets to hold collections of data organized into versions
#    https://docs.snowflake.com/en/developer-guide/snowpark-ml/reference/latest/dataset



# 🦜 Datasets are Snowflake schema-level objects specially designed for machine learning 
#    workflows. Snowflake Datasets hold collections of data organized into versions, where 
#    each version holds a materialized snapshot of your data with guaranteed immutability, 
#    efficient data access, and interoperability with popular deep learning frameworks.



# Generate a random version number, it must start with letter or digit, and followed by 
# letter, digit, '_', '-' or '.'. The length limit is 128.
version_number = 'v' + datetime.now().strftime( '%Y-%m-%d_%H-%M-%S' )



# Materialize DataFrame contents into a Dataset and version control it.
client_features_ds = dataset.Dataset.create(
    sf_session,
    name = db_name + '.' + schema_name + '.' + 'CLIENT_FEATURES_DS',
    exist_ok = True,
)



client_features_dsv = client_features_ds.create_version(
    version = version_number,
    input_dataframe = sf_session.table( client_features_table_name ).drop( ['ID', 'FIRST_NAME', 'LAST_NAME'] ),
)



# Load into a Snowpark Dataframe to display.
client_features_df = client_features_dsv.read.to_snowpark_dataframe()
st.subheader( 'This is in effect, simply the data from the CLIENT_FEATURES table...' )
st.dataframe( client_features_df.first(10) )



# Let's see what Datasets are in the Account now.
st.subheader( '...but logged as a Dataset so it can now be for example version controlled.' )
st.dataframe( sf_session.sql( 'show datasets in account' ) )

## When should you use Datasets?

You should use Snowflake Datasets when:
* You need to manage and version large datasets for reproducible machine learning model training and testing.
* You want to leverage Snowflake’s scalable and secure data storage and processing capabilities.
* You need fine-grained file-level access and/or data shuffling for distributed training or data streaming.
* You need to integrate with external machine learning frameworks and tools.

❗❗❗ Because we used the Snowflake ML API to engineer the data we can see the lineage, from the raw table to the feature engineered table, let's go and take a look in the Lineage UI in Snowsight.

In [ ]:
# Features shown here:
# 📘 Common Python packages working with Notebooks
#    https://repo.anaconda.com/pkgs/snowflake/
#    https://www.anaconda.com/partners/snowflake



# 🦜 In order to use your data you need to understand it, to do this included with
#    you need to visualise it, Snowpark includes a number of popular open source 
#    third-party Python packages that are built and provided by Anaconda are made 
#    available to use out of the box inside Snowflake virtual warehouses.



# Get the dataframe we created earlier and check it for correlated features using
# the correlation function in snowflake.ml.modeling.metrics.
corr_clients_df = metrics.correlation( 
    df = client_features_dsv.read.to_snowpark_dataframe()
)



# Generate a heatmap with the features so we can see which have the most effect.
mask = np.triu(
    np.ones_like(
        corr_clients_df, 
        dtype=bool
    )
)



plt.figure( figsize=( 3, 3 ) )
plt.rc( 'xtick', labelsize=5 )
plt.rc( 'ytick', labelsize=5 )
heatmap = sns.heatmap(
    corr_clients_df, 
    mask=mask, 
    cmap="YlGnBu", 
    annot=False, 
    vmin=-1, 
    vmax=1
)



# 🍾 Access more than 2,000 Python packages directly from within Notebooks, 
#    Streamlit, Stored Procedures, Python Worksheets, etc.

In [ ]:
# Features shown here:
# 📘 Streamlit
#    https://docs.snowflake.com/en/developer-guide/streamlit/about-streamlit



# 😕 However, I've always found matplotlib's charts a little fixed, not very 
#    interactive and to be honest not very attractive. What could we use 
#    instead? Streamlit of course! Streamlit is great as it allows you to 
#    build interactive UI's quickly and easily.



# Get our client features data from the Dataset and summarise it.
client_features_summed_df = client_features_dsv.read.to_snowpark_dataframe().to_pandas().groupby(
    ['SMOKER_ENCODED', 'AGE', 'CHILDREN']
).aggregate(
    {'TOTAL_CLAIMS':['min', 'max', 'mean', 'count']}
).reset_index()



# After summarising, munge it a bit to get rid of multi-level data structures, etc.
client_features_summed_df.columns = client_features_summed_df.columns.map( '_'.join )



# Lets create some UI components that will allow users to choose their own measures
# and groupings to plot on a chart.

# 🍾 This means you as a developer can build self-service into your work, reducing
#    your workload and making you, and your customers, more productive.

col1, col2 = st.columns(2)

with col1:
    bar_colour_scheme = st.selectbox( 
        label = 'Colour Scheme',
        options = ('category20', 'category20', 'tableau10', 'tableau20', 'teals', 'plasma', 'rainbow')
    )
    
    measure_to_display_dict = {
        'Min': 'TOTAL_CLAIMS_min', 
        'Max': 'TOTAL_CLAIMS_max', 
        'Mean': 'TOTAL_CLAIMS_mean', 
        'Count': 'TOTAL_CLAIMS_count'
    }
    
    measure_to_display = st.radio(
        label = 'Choose a measure to plot', 
        options = ['Min', 'Max', 'Mean', 'Count'], 
        horizontal = True, 
    )

with col2:
    color_to_display_dict = {
        'Smoker': 'SMOKER_ENCODED_', 
        'Children': 'CHILDREN_'
    }
    
    color_to_display = st.selectbox(
        label = 'Choose a grouping to plot', 
        options = ['Smoker', 'Children'], 
    )



# Now we can build a chart object.
chart = alt.Chart(client_features_summed_df).mark_circle(size=60).encode(
    x=alt.X(
        'AGE_:O', 
        axis=alt.Axis( title='Age', grid=True, format=',.0f' )
    ),
    y=alt.Y(
        f'{measure_to_display_dict[measure_to_display]}:Q', 
        axis=alt.Axis( title=f'{measure_to_display}', grid=True, format=',.0f' )
    ),
    color=alt.Color(
        f'{color_to_display_dict[color_to_display]}:O', 
        legend=alt.Legend( orient="bottom", columns=4, title=f'{color_to_display}' ),
        scale=alt.Scale( scheme=bar_colour_scheme ),
    ),
    tooltip = [
        alt.Tooltip( 'AGE_', title='Age' ),
        alt.Tooltip( 'SMOKER_ENCODED_', title='Smoker?' ),
        alt.Tooltip( 'TOTAL_CLAIMS_min', title='Min', format=',.0f' ),
        alt.Tooltip( 'TOTAL_CLAIMS_max', title='Max', format=',.0f' ),
        alt.Tooltip( 'TOTAL_CLAIMS_mean', title='Mean', format=',.0f' ),
        alt.Tooltip( 'TOTAL_CLAIMS_count', title='Count' ),
    ]
).interactive()



# Then display it.
st.altair_chart( chart, use_container_width=True )

In [ ]:
# Features shown here:
# 📘 Snowpark ML Modeling API for model development and training
#    https://docs.snowflake.com/en/developer-guide/snowflake-ml/modeling



# 🦜 Now our data is sorted, lets build and train a machine learning model that will be
#    used to predict the amount of claims a customer may make.



# Split the data into train and test sets
train_df, test_df = client_features_dsv.read.to_snowpark_dataframe().random_split( weights=[0.9, 0.1], seed=0 )



# Define the XGBoost model, including distributed Hyperparameter Tuning.
tuned_xgboost_model = model_selection.GridSearchCV(
    
    # Choose an algorithm to use by the model.
    estimator = xgboost.XGBRegressor(),

    param_grid = {
        # This is where the Hyperparameter Tuning occurs. The following will effectively give us
        # four differently trained model variations, and use the one with the lowest MAPE, see 
        # scoring below.
        'n_estimators':[100, 200],
        'learning_rate':[0.1, 0.2],
    },

    # Use all cores available on the Warehouse.
    n_jobs = -1,

    # This is what will be used by the model when called to decide which parameter configuration to use.
    scoring = 'neg_mean_absolute_percentage_error',

    # Define inputs and outputs.
    input_cols = ['AGE', 'GENDER_ENCODED', 'CHILDREN', 'BMI', 'SMOKER_ENCODED'],
    label_cols = ['TOTAL_CLAIMS'],
    output_cols = ['TOTAL_CLAIMS_PREDICTION'],
    
)



# Fit / train the pipeline to the training data
fitted_xgboost_model = tuned_xgboost_model.fit( train_df )



# 🍾 Again, we're using common ML libraries such as sklearn which you already know how to use, 
#    saving you the time and effort of learning something new.

In [ ]:
# Features shown here:
# 📘 Snowpark ML Modeling API for model development and training



# 🦜 Before we make it available for everyone to use lets test it.



# Test it locally using the test data we created above.
predicted_claims = fitted_xgboost_model.predict( test_df )
st.dataframe( predicted_claims.to_pandas().astype("float64") )

In [ ]:
# Features shown here:
# 📘 Snowflake Model Registry to securely manage models and their metadata in Snowflake, regardless of origin.
#    https://docs.snowflake.com/en/developer-guide/snowflake-ml/model-registry/overview



# 🦜 Great, it works, now lets log it into the Snowflake Model Registry so anyone can use it.
#    The Snowflake Model Registry lets you securely manage models and their metadata in Snowflake, 
#    regardless of origin. Models can have multiple versions, and you can designate a version as the default.



# Opening the registry so we can use it to add new models and obtain references to existing models.
model_registry = registry.Registry( sf_session )



# Create a version of the model, if you leave out the version name we create one for you.
fitted_xgboost_model_version = model_registry.log_model(
    model = fitted_xgboost_model,
    model_name = db_name + '.' + schema_name + '.' + 'PREDICT_CLAIMS',
    # version_name = 'v1', # You don't need to specify a version as one will be automatically generated.
    comment = 'Model used to predict life insurance claims.',
)

In [ ]:
# Lets see what we logged. Beacuse models are logged in Snowflake, you can see them from common
# views like the INFORMATION_SCHEMA, as well as show commands.
st.subheader( 'Models Logged' )
logged_models = sf_session.sql( 'show models in account;' )
st.dataframe( logged_models )



st.subheader( 'Models Versions Logged in PREDICT_CLAIMS' )
predict_claims_model_versions = sf_session.sql( 'show versions in model ' + db_name + '.' + schema_name + '.' + 'PREDICT_CLAIMS;' )
st.dataframe( predict_claims_model_versions )



# 🍾 The model is now logged in a central, secure location which protects both your 
#    data and your intellectual property in one place.

In [ ]:
# Features shown here:
# 📘 Snowflake Model Registry to securely manage models and their metadata in Snowflake, regardless of origin.
#    https://docs.snowflake.com/en/developer-guide/snowflake-ml/model-registry/overview



# Now lets test the model by calling it via the Model Registry.
xgboost_model_prediction = fitted_xgboost_model_version.run( test_df, function_name='predict' )

st.dataframe( xgboost_model_prediction.select(['AGE', 'SMOKER_ENCODED', 'TOTAL_CLAIMS', 'TOTAL_CLAIMS_PREDICTION']) )

So what now? We have a model but one of the questions asked was "What are the options to surface the output to stakeholders?"

There are a few ways we can do that, you can use it in other parts of Snowflake like Worksheets or in tools like Streamlit, lets take a look at how you do that now.

In [ ]:
# USEFUL RESOURCES
# ================

# Medium:
# https://medium.com/snowflake/getting-started-with-snowpark-model-registry-131e5a2783c4
# https://medium.com/@soonmo.seong/how-to-use-snowpark-ml-model-registry-in-snowflake-09f93312241b
# https://medium.com/snowflake/productionize-your-ml-workflow-using-snowflake-task-dag-apis-8470aa33172c
# https://medium.com/snowflake-engineering/snowflake-python-api-the-key-to-modern-data-pipeline-creation-2ab864c7aa5c

# Documentation:
# https://docs.snowflake.com/en/developer-guide/snowpark-ml/model-registry/overview

# Snowflake Labs:
# Tyler's > https://github.com/Snowflake-Labs/sfguide-getting-started-dataengineering-ml-snowpark-python/blob/main/Snowpark_For_Python_ML.ipynb
# https://github.com/Snowflake-Labs/sf-samples/blob/main/samples/ml/mlops_using_model.ipynb
# https://github.com/Snowflake-Labs/sfguide-intro-to-machine-learning-with-snowflake-ml-for-python/blob/main/1_sf_nb_snowflake_ml_data_ingest.ipynb
# https://github.com/Snowflake-Labs/sfguide-intro-to-machine-learning-with-snowflake-ml-for-python/blob/main/2_sf_nb_snowflake_ml_feature_transformations.ipynb
# https://github.com/Snowflake-Labs/sfguide-deploying-custom-models-snowflake-model-registry/blob/main/2_create_and_deploy_custom_model.ipynb

# Quickstarts
# https://quickstarts.snowflake.com/guide/deploying_custom_models_to_snowflake_model_registry/index.html?index=..%2F..index#0

# Internal
# https://github.com/michaelgorkow/snowflake_simple_ml/blob/main/demo_notebook.ipynb

In [ ]:
-- What models have been created?
-- show models in account;
-- show versions in model PREDICT_CLAIMS;



-- Drop a model version, you can do this in Python as well but we'll use SQL here.
-- alter model PREDICT_CLAIMS drop version POPULAR_CATFISH_2;



-- Drop a Dataset.
-- show datasets in account;
-- drop dataset INSURANCE.PUBLIC.CLIENT_FEATURES_DS;